## 추가해야하는 내용

In [1]:
# Check core SDK version number
import azureml.core
import os

print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


# 1. Azure ML Service 작업 영업에 연결 

#### Notebook이 실행되는 경로 내 Azure ML Service Workspace 접속 정보를 가진 config 파일이 있어야한다.

 - 테스트 배포 시에는 'data/config_dev.json' 사용
 - 실제 배포 시에는 'data/config.json' 사용

In [3]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c8b9cf2c-f80b-4c36-912f-74979c640070")

ws = Workspace(subscription_id="1f154107-bef0-481f-a741-68e7ba34affe",
               resource_group="lsmc-prd-rg",
               workspace_name="lsmc-dev-mlws",
               auth=interactive_auth)

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
from azureml.core import Workspace

ws = Workspace.from_config('deployment/config/config_dev.json')
print('기계학습이름:'+ws.name, '리소스그룹이름:'+ws.resource_group, '리전이름:'+ws.location, '구독ID:'+ws.subscription_id, sep = '\n')

기계학습이름:lsmc-dev-mlws
리소스그룹이름:lsmc-prd-rg
리전이름:koreacentral
구독ID:1f154107-bef0-481f-a741-68e7ba34affe


# 2. 모델 등록

#### 등록할 모델의 버전을 명시해준다. 

In [5]:
model_version = "03_050_02"
file_name = '{}.pickle'.format(model_version)
print(file_name)

03_050_02.pickle


## Description 수정해아함!!!

#### 모델을 등록해준다. description에 현재 모델에 대한 간단한 설명을 추가할 수 있다.

 - 실행하면, [Azure Portal > 기계 학습 > 기계학습이름(위에서 확인) > 모델] 에서 확인 가능함

In [6]:
from azureml.core.model import Model

#library_version = "DL"+sklearn.__version__.replace(".","x")
model = Model.register(model_path = "deployment/data/pkl/{}".format(file_name),
                       model_name = model_version,
                       tags = {'area': "IoT Edge", 'type': "azureml-automl"},
                       description = "AzureML SDK upgrade & Model upgrade",
                       workspace = ws)

Registering model 03_050_02


# 3. 이미지 등록

#### 등록 된 모델 이름을 사용 하 여 모델 파일에 대 한 경로를 반환

In [7]:
from azureml.core.model import Model
model = Model.get_model_path(model_version, _workspace=ws)
model

'azureml-models\\03_050_02\\9\\03_050_02.pickle'

#### 사용되는 package 선언. Edge 환경에서 해당 package를 사용되기 때문에 channel 과 필요한 package명을 선언 해주어야 한다. 

In [8]:
pip_packages = ["azureml-defaults",
                "azureml==0.2.7",
                "azureml-core==1.24.0",
                "azureml-sdk==1.24.0",
                "azureml-automl-core==1.24.0",
                "azureml-automl-runtime==1.24.0",
                "azure-storage-blob==12.6.0",
                "numpy==1.18.5",
                "pandas==0.25.3",
                "sqlalchemy==1.3.21",
                "h5py==2.10.0",
                "tqdm==4.54.0",
                "obspy==1.2.2",
                "mysql-connector-python==8.0.18",
                "joblib==0.14.1",
                "packaging==20.7",
                "xgboost==0.90",
                "noisereduce==1.1.0"]

lib_config_load = ['pyyaml']
lib_clfs = ["gxx_linux-64",
            "gcc_linux-64",
            "librosa==0.8.0"]
#lib_clfs = ['tensorflow==1.1']
conda_packages = lib_clfs + lib_config_load

print('pip packages:', pip_packages)
print('conda packasges:', conda_packages)

pip packages: ['azureml-defaults', 'azureml==0.2.7', 'azureml-core==1.24.0', 'azureml-sdk==1.24.0', 'azureml-automl-core==1.24.0', 'azureml-automl-runtime==1.24.0', 'azure-storage-blob==12.6.0', 'numpy==1.18.5', 'pandas==0.25.3', 'sqlalchemy==1.3.21', 'h5py==2.10.0', 'tqdm==4.54.0', 'obspy==1.2.2', 'mysql-connector-python==8.0.18', 'joblib==0.14.1', 'packaging==20.7', 'xgboost==0.90', 'noisereduce==1.1.0']
conda packasges: ['gxx_linux-64', 'gcc_linux-64', 'librosa==0.8.0', 'pyyaml']


In [9]:
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core import Environment

conda_deps  = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
conda_deps.add_channel('conda-forge')
conda_deps.add_channel('defaults')

myenv = Environment(name='myenv')
myenv.python.conda_dependencies = conda_deps
myenv.docker.base_image = 'mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04'
# myenv.docker.base_image = 'mcr.microsoft.com/azureml/base:openmpi3.1.2-ubuntu18.04'


# with open("data/myenv.yml","w") as f:
#     f.write(myenv.serialize_to_string())

# 4. score_iot.py 파일(전처리 및 ML 판정 수행 로직 실행 스크립트) 생성

 ### 아래 cell 이 실행되면 score_iot.py에 실행 스크립트가 생성됨

### score_iot_test.ipynb에 해당 스크립트의 내용을 복사하여, 실행 시 에러가 발생하지 않는지 꼭 확인!

In [10]:
import _locale
_locale._getdefaultlocale = (lambda *args: ['en_US', 'utf8'])

In [11]:
%%writefile src/score_iot.py


# For Edge
import json
import yaml
import time
import os
import re
import pickle
import shutil
from datetime import datetime
from datetime import timedelta
from time import sleep
from azureml.core.model import Model

# For Clf
import h5py
import pandas as pd
from classify import Classifier
from preprocess_signal_data import Run_
from pkgs.pipelines import FeatureDeployed as Featurizing
from pkgs.utils import read_json


def init_preprocessor(feature_params, Featurizing, test_no, input_type="ReadData"):
    # set preprocess class
    preproc = Run_(
        feature_params=feature_params, Featurizing=Featurizing, test_no=test_no
    )
    if input_type == "ReadData":
        preproc.pl.steps = [
            (step_name, step)
            for step_name, step in preproc.pl.steps
            if step.name != "ReadData"
        ]

    return preproc


def score(
    preproc,
    input_values,
    feature_names,
    column_names,
    cut_off,
    rule_model_params,
    mode,
    input_name="ReadData",
    parallel=1,
    verbose=0,
):
    # Preprocess
    df = preproc.pl.run(
        input_name=input_name,
        input_values=[input_values],
        feature_names=feature_names,
        parallel=parallel,
        verbose=verbose,
    )
    df = pd.DataFrame(df[0])
    df["TEST_NO"] = [0,1,2]
    df_features = df[column_names]

    # Predict
    df["PROB"] = loaded_model.predict_proba(df_features)[:,1]
    
    # Classify
    df["ML_R"] = df.PROB >= cut_off
    df["ML_R"] = df.ML_R.astype(int)
    df = Classifier.classify_all(
        df=df[df.TEST_NO.isin(preproc.featurizer.params["test_no"])],
        rule_cutoffs=rule_model_params,
        mode=mode,
    )
    return df, df_features


def init():
    global preproc, loaded_model, model_version, column_names, line, cut_off, rule_model_params, ml_model_params, model_path, error_dir
    # 수정필요  --------------------------------------------------------
    model_version = "03_050_02"  # model_name 입력
    file_name = "{}.pickle".format(model_version)
    edge_config = "/home/data/edge_config.yml"
#     file_name = "deployment/data/pkl/{}.pickle".format(model_version)
#     edge_config = "deployment/config/edge_config.yml"
    feature_params = read_json("src/config/featureD050_parameters.json")
    column_names = read_json("src/config/model_03_050_02_features.json")["feature_names"]
    # ----------------------------------------------------------------

    # load line info
    with open(edge_config, "r") as stream:
        try:
            edge_config = yaml.load(stream, Loader=yaml.BaseLoader)
            line = edge_config["config"]["line"]["name"]
            cut_off = float(edge_config["config"]["param"]["cutoff"])
            rule_model_params = edge_config["config"]["param"]["rule_model"]
            ml_model_params = edge_config["config"]["param"]["ml_model"]

        except yaml.YAMLError as exc:
            print("line config error: ", exc)

    # set preprocess class
    for param_, dic_ in ml_model_params.items():
        for key_, value_ in dic_.items():
            feature_params[param_][key_] = float(value_)
    preproc = init_preprocessor(
        feature_params=feature_params, Featurizing=Featurizing, test_no=[0, 1, 2]
    )

    # load model
    model_path = Model.get_model_path(file_name)
    with open(model_path, "rb") as f:
        loaded_model = pickle.load(f)
    
    error_dir = "/home/data/error_file/"
    try:
        os.makedirs(error_dir, mode=777)
    except:
        pass


def run(input_json):
    print("\n", "mlmodule start")
    print("\n", datetime.now() + timedelta(hours=9), "\n")
    # for test#
    input_json = json.loads(input_json)
    print("\n", "json loaded", "\n")
    print(input_json, "\n")
    mltime = datetime.now() + timedelta(hours=9)
    chtime = input_json["chtime"]
    print("chtime : ", chtime)
    ct = datetime.strptime(
        chtime.replace("T", " ").split("+")[0][:-1], "%Y-%m-%d %H:%M:%S.%f"
    )
    diff = mltime - ct
    input_json["chtime"] = str(ct)
    input_json["mltime"] = str(mltime)
    input_json["etime_ch"] = diff.seconds + diff.microseconds / 1e6

    # file load
    init_time = time.time()
    # input_json = json.loads(input_json)
    input_path = input_json["path"]
    print("\n", input_path)

    for attempt in range(3):
        try:
#             input_raw = pr.ReadDataBlob(input_path)
            print("Attempt: " + str(attempt + 1) + "  Time: " + str(datetime.now()))
            with h5py.File(input_path, "r") as f:
                tmp = f["Raw"][:]
            input_raw = pd.DataFrame(tmp, columns=[0, 1, 2])
            print("\n", "hdf loaded")

        except Exception as ex:
            if attempt == 2:
                ex_message = str(ex)

            sleep(0.02)
            continue

        break

    else:
        input_json["b"] = 2
        input_json["prob"] = 2
        input_json["error"] = ex_message
        input_json["etime"] = time.time() - init_time

        result_json = [json.dumps(input_json)]
        print("*" * 5, " ", "LOAD ERROR", " ", "*" * 5)
        print(result_json)
        try:
            shutil.copy(input_path, error_dir)
        except:
            pass

    # for test#
    # diff = (datetime.now()+ timedelta(hours=9))-mltime
    # input_json['etime_load'] = diff.seconds + diff.microseconds/1E6
    input_json["etime_load"] = time.time() - init_time

    ### json insert
    input_json["cutoff"] = cut_off
    input_json["TRHD_NM_SET"] = "/".join([key for key in rule_model_params.keys()] + [f"{key_}-{param}" for key_, dict_ in ml_model_params.items() for param, value in dict_.items()])
    input_json["TRHD_VAL_SET"] = "/".join(
        [str(value) for value in rule_model_params.values()]+[str(value) for key_, dict_ in ml_model_params.items() for param, value in dict_.items()]
    )
    input_json["FTUR_NM_SET"] = "/".join(
        [
            "/".join([col + "_TEST_N1" for col in column_names]),
            "/".join([col + "_TEST_N2" for col in column_names]),
            "/".join([col + "_TEST_N3" for col in column_names]),
        ]
    )
    input_json["lid"] = line
    input_json["v"] = model_version

    try:
        filename = input_path.split("/")[-1]
        filename = filename.strip("Data\\").split("_")
        input_json["bc"] = "_".join(filename[0:2])
        input_json["dtfull"] = filename[2][:-3]

    except Exception as ex:  # 에러 종류
        input_json["bc"] = "ERROR"
        input_json["dtfull"] = str(datetime.now())
        input_json["r"] = 2
        input_json["prob"] = 2
        input_json["error"] = str(ex)
        input_json["etime"] = time.time() - init_time

        result_json = [json.dumps(input_json)]
        print("*" * 5, " ", "FILENAME ERROR", " ", "*" * 5)
        print(result_json)
        try:
            cmd = "cp " + input_json["path"] + " " + error_dir
            os.system(cmd)
        except:
            pass

    try:
        df, df_features = score(
            preproc=preproc,
            input_values=input_raw,
            feature_names=preproc.featurizer.params["feature_names"],
            column_names=column_names,
            cut_off=cut_off,
            rule_model_params=rule_model_params,
            mode="or",
            input_name="ReadData",
            parallel=1,
            verbose=0,
        )
        
        input_json["r"] = int(df.R.values[0])
        input_json["prob"] = float(df.PROB.values[0])
        input_json["error"] = str(df.TEST_NG_R.values[0])
        input_json["etime"] = time.time() - init_time
        input_json["FTUR_VAL_SET"] = "/".join(
            [str(value) for value in df_features.values.flatten()]
        )

        result_json = [json.dumps(input_json)]
        print(result_json)

    except Exception as ex:  # 에러 종류
        input_json["r"] = 2
        input_json["prob"] = 2
        input_json["error"] = str(ex)
        input_json["etime"] = time.time() - init_time
        input_json["FTUR_VAL_SET"] = ""

        result_json = [json.dumps(input_json)]
        print("*" * 5, " ", "ERROR", " ", "*" * 5)
        print(result_json)
        try:
            cmd = "cp " + input_json["path"] + " " + error_dir
            os.system(cmd)
        except:
            pass

    return result_json


Overwriting src/score_iot.py


In [12]:
import os
os.getcwd()

'C:\\Users\\Nam\\Documents\\Jupyter Test\\ML Module Test'

# 5. Azure MLWS에 이미지 업로드

In [13]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score_iot.py", environment=myenv, source_directory="src")
package_service = Model.package(workspace=ws,
                                models=[model],
                                image_name="mc-mlmodule",
                                image_label=17,
                                inference_config=inference_config)

ModelNotFound: Model with id azureml-models\03_050_02\9\03_050_02.pickle not found in provided workspace



Registering model 03_050_02.pickle


In [14]:
package_service.wait_for_creation(show_output=True)

2021/03/23 06:04:30 Downloading source code...
2021/03/23 06:04:32 Finished downloading source code
2021/03/23 06:04:32 Creating Docker network: acb_default_network, driver: 'bridge'
2021/03/23 06:04:33 Successfully set up Docker network: acb_default_network
2021/03/23 06:04:33 Setting up Docker configuration...
2021/03/23 06:04:34 Successfully set up Docker configuration
2021/03/23 06:04:34 Logging in to registry: lsmcdevmlwsfe4aa203.azurecr.io
2021/03/23 06:04:35 Successfully logged into lsmcdevmlwsfe4aa203.azurecr.io
2021/03/23 06:04:35 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/03/23 06:04:35 Scanning for dependencies...
2021/03/23 06:04:35 Successfully scanned dependencies
2021/03/23 06:04:35 Launching container with name: acb_step_0
Sending build context to Docker daemon  67.07kB
Step 1/18 : FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04@sha256:a1b514f3ba884b9a7695cbba5638933ddaf222e8ce3e8c81e8cdf8

openh264-2.1.0       | 1.5 MB    | ########## | 100% 
librosa-0.8.0        | 145 KB    | ########## | 100% 
freetype-2.10.4      | 901 KB    | ########## | 100% 
libpng-1.6.37        | 364 KB    | ########## | 100% 
pysoundfile-0.10.3.p | 23 KB     | ########## | 100% 
libffi-3.2.1         | 52 KB     | ########## | 100% 
readline-7.0         | 387 KB    | ########## | 100% 
tornado-6.0.4        | 650 KB    | ########## | 100% 
urllib3-1.25.11      | 93 KB     | ########## | 100% 
cffi-1.14.0          | 225 KB    | ########## | 100% 
libsndfile-1.0.29    | 534 KB    | ########## | 100% 
chardet-3.0.4        | 197 KB    | ########## | 100% 
pycparser-2.20       | 94 KB     | ########## | 100% 
binutils_impl_linux- | 8.6 MB    | ########## | 100% 
python-3.6.2         | 27.0 MB   | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... 
done
Executing transaction: ...working... 
done
  Installing build dependencies: started
  Installing build de


  Created wheel for obspy: filename=obspy-1.2.2-cp36-cp36m-linux_x86_64.whl size=21694285 sha256=ff92dcd513d3f340afc51c6f02887e8617a4b6ab42bf6fb2ceafc8a042bf3d2e
  Stored in directory: /root/.cache/pip/wheels/e5/26/f8/739635bd6ce26550743eac39d0a4fd5dfb01b16b4d3a644ebc
  Created wheel for noisereduce: filename=noisereduce-1.1.0-py3-none-any.whl size=7607 sha256=c9c77fb06e5add1447d1f803792b029e3e0ce9b82708dde41fbccabe0e521e5d
  Stored in directory: /root/.cache/pip/wheels/a9/d4/51/49c94295d30569d64c83dc11dce56166173635c85d060c9859
  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3924 sha256=bf42eb589294fc0300032049ac004024a5ce7603bbe3c358517d515126e6d00a
  Stored in directory: /root/.cache/pip/wheels/e2/1d/52/535a274b9c2ce7d4064838f2bdb62013801281ef7d7f21e2ee
  Created wheel for smart-open: filename=smart_open-1.9.0-py3-none-any.whl size=73087 sha256=c481819607e47d76bccdb0dd9adcc307c676cb6a45b4f86ee727bb4ae7676e0f
  Stored in directory: /root/.cach

    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1

  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2

  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4

#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_882aeb18bd4696f212b5c79bda0dd217
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 2cbd1368dc90
 ---> 681b3c0ae338
Step 9/18 : ENV PATH /azureml-envs/azureml_882aeb18bd4696f212b5c79bda0dd217/bin:$PATH
 ---> Running in b281de2a79fb
Removing interm

2021/03/23 06:20:25 Downloading source code...
2021/03/23 06:20:27 Finished downloading source code
2021/03/23 06:20:28 Creating Docker network: acb_default_network, driver: 'bridge'
2021/03/23 06:20:31 Successfully set up Docker network: acb_default_network
2021/03/23 06:20:31 Setting up Docker configuration...
2021/03/23 06:20:32 Successfully set up Docker configuration
2021/03/23 06:20:32 Logging in to registry: lsmcdevmlwsfe4aa203.azurecr.io
2021/03/23 06:20:33 Successfully logged into lsmcdevmlwsfe4aa203.azurecr.io
2021/03/23 06:20:33 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/03/23 06:20:33 Launching container with name: acb_step_0
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /acb/home/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-

 ---> Running in 14ecb38d53d4
Removing intermediate container 14ecb38d53d4
 ---> d8f408228d4e
Step 7/7 : CMD ["runsvdir","/var/runit"]
 ---> Running in f5172f8a3253
Removing intermediate container f5172f8a3253
 ---> 109cf1c9f37e
Successfully built 109cf1c9f37e
Successfully tagged lsmcdevmlwsfe4aa203.azurecr.io/azureml/azureml_beae5e83137d4ca411feb401bf73b506:latest
Successfully tagged lsmcdevmlwsfe4aa203.azurecr.io/azureml/azureml_beae5e83137d4ca411feb401bf73b506:1
2021/03/23 06:22:57 Successfully executed container: acb_step_1
2021/03/23 06:22:57 Executing step ID: acb_step_2. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/03/23 06:22:57 Pushing image: lsmcdevmlwsfe4aa203.azurecr.io/azureml/azureml_beae5e83137d4ca411feb401bf73b506:1, attempt 1
The push refers to repository [lsmcdevmlwsfe4aa203.azurecr.io/azureml/azureml_beae5e83137d4ca411feb401bf73b506]
9a21c9c708a4: Preparing
3b25a40394f4: Preparing
972ff9dabb0a: Preparing
5c353fbff8e1: Preparing
042f1

In [15]:
print(package_service.state)

Succeeded


In [16]:
print(package_service.location)

lsmcdevmlwsfe4aa203.azurecr.io/mc-mlmodule@sha256:6f4f8dfb493bf31114b67995efff6bccd9211ffe6f96e898762b17514917979f
